# Setup

In [1]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 4.0 MB/s eta 0:00:00


In [2]:
# Helper functions
from openai import AsyncOpenAI, OpenAI
from agents import (
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled
)
from agents.items import ToolCallItem, ToolCallOutputItem, MessageOutputItem, ItemHelpers
from google.colab import userdata
import nest_asyncio
nest_asyncio.apply()

# Gemini API configuration
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = userdata.get("GOOGLE_API_KEY")
MODEL_NAME = "gemini-2.0-flash"

# Create a custom client for Gemini
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

# Configure the agents library to use Gemini
set_default_openai_client(client=client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)


def print_conversation(result):

    # Print the original input if it exists
    if hasattr(result, "input"):
        print("Input:", result.input)

    # Iterate over each item generated during the run
    for msg in result.new_items:
        if isinstance(msg, ToolCallItem):
            # Print tool call details using the raw item attributes
            print(f"Tool Call: {msg.raw_item.name}({msg.raw_item.arguments})")
        elif isinstance(msg, ToolCallOutputItem):
            # Print tool call output
            print(f"Tool Output: {msg.output}")
        elif isinstance(msg, MessageOutputItem):
            # Extract the text using the helper function
            text = ItemHelpers.text_message_output(msg)
            # Determine the role from the raw item; default to "message" if not provided
            role = getattr(msg.raw_item, "role", "message").capitalize()
            print(f"{role}: {text.strip()}")
        else:
            print(f"Unknown message type: {msg}")



async def run_agent(agent, message):
    result = await Runner.run(agent, message)
    print_conversation(result)
    return result



# 1. Agente básico con calculadora

1. Define el agente con nombre e instrucciones.

1. Ejecuta el agente y pidele que haga una suma. Revisa el output del agente.

2. Añade dos tools nuevas al agente, una de multiplicación y una de división.

3. Pide al Agente que haga una multiplicación y una división.

In [3]:
from agents import function_tool, Agent, Runner

# Import MODEL_NAME
from __main__ import MODEL_NAME

@function_tool
def sumar(a: int, b: int) -> str:
    result = a + b
    return str(result)

# Definimos el Agente
agent_matematico = Agent(
    model=MODEL_NAME,
    name="Matematico",
    instructions="Eres un calculador inteligente.",
    tools=[sumar],
)

In [4]:
# Ejecutamos el agente
mensaje_input = "¿Cuánto es 5+5?"
result = await run_agent(agent_matematico, mensaje_input)

Input: ¿Cuánto es 5+5?
Tool Call: sumar({"b":5,"a":5})
Tool Output: 10
Assistant: El resultado de 5 + 5 es 10.


# Agente para reservas de hoteles

1. Lee la base de datos de reservas `reservas_db`.


2. Usa el agente para consultar si hay disponiblidad el 19 de Abril de 2025.


3. Añade una nueva tool para crear reservas.

  ```
  def crear_reserva(fecha_entrada: date, fecha_salida: date, cliente: str) -> str:
    # crear nueva reserva
    ...
    reservas_db.append(nueva_reserva)
    return f"La reserva ha sido creada con éxito"
  ```

4. Añade la tool al Agente y instrucciones de cómo usarla.

5. Prueba de añadir una reserva para el 25 de Marzo. Para mensajes consecutivos en una misma conversación, usa el siguiente código:

  ```
  mensaje_input = "Mi nombre es Marco Gómez"
  conversation_history = result.to_input_list()
  nuevo_input = conversation_history + [{"role": "user", "content": mensaje_input}]
  ```


6. Comprueba que la nueva reserva ha sido creada en `reservas_db`.

In [5]:
from datetime import date

# Definimos un estado inicial de disponibilidad de habitaciones
reservas_db = [
    {"num_reserva": 1, "fecha_entrada": date(2025, 4, 1), "fecha_salida": date(2025, 4, 5), "cliente": "Juan Pérez"},
    {"num_reserva": 2, "fecha_entrada": date(2025, 4, 18), "fecha_salida": date(2025, 4, 20), "cliente": "Pedro López"},
]

In [6]:
@function_tool
def consultar_reservas(fecha_entrada: date, fecha_salida: date) -> str:

    for reserva in reservas_db:
        # Verificar si hay solapamiento de fechas
        if (fecha_entrada <= reserva["fecha_salida"] and fecha_salida >= reserva["fecha_entrada"]):
            return f"La habitación no está disponible entre las fechas {fecha_entrada} y {fecha_salida}"

    return f"La habitación está disponible entre las fechas {fecha_entrada} y {fecha_salida}"


today = date.today()

instructions = f"""Eres un asistente inteligente que gestiona reservas para el Hotel del Sol.
El hotel tiene una sola habitación. Comprueba si hay disponibilidad para las fechas indicadas. Hoy es {today}.
"""

agente_reservas = Agent(
    model=MODEL_NAME,
    name="Reservas",
    instructions=instructions,
    tools=[consultar_reservas],
)

In [7]:
# Ejecutamos el agente
mensaje_input = "Hola quería..."
result = await run_agent(agente_reservas, mensaje_input)

Input: Hola quería...
Assistant: ¿En qué puedo ayudarte?


In [8]:
# 19 de Abril de 2025
mensaje_input =  "Hay disponiblidad el 19 de Abril de 2025?"
result = await run_agent(agente_reservas, mensaje_input)

Input: Hay disponiblidad el 19 de Abril de 2025?
Assistant: ¿Cuántas noches te gustaría reservar?


In [9]:
def crear_reserva(fecha_entrada: date, fecha_salida: date, cliente: str) -> str:
 nueva_reserva = [
    {"num_reserva": 3, "fecha_entrada": date(2025, 4, 1), "fecha_salida": date(2025, 5, 5), "cliente": "Ali C."},
    {"num_reserva": 4, "fecha_entrada": date(2025, 5, 18), "fecha_salida": date(2025, 5, 20), "cliente": "Laure M."},
]

 reservas_db.append(nueva_reserva)
 return f"La reserva ha sido creada con éxito"

today = date.today()

instructions = f"""Eres un asistente inteligente que gestiona reservas para el Hotel del Sol.
El hotel tiene una sola habitación. Comprueba si hay disponibilidad para las fechas indicadas. Hoy es {today}.
"""

agente_reservas = Agent(
    model=MODEL_NAME,
    name="Reservas",
    instructions=instructions,
    tools=[consultar_reservas],
)

Prueba de añadir una reserva para el 25 de Marzo

In [10]:
# Ejecutamos el agente
mensaje_input = "Hola quería hacer una reserva"
result = await run_agent(agente_reservas, mensaje_input)

Input: Hola quería hacer una reserva
Assistant: Ok. ¿Cuáles son las fechas de entrada y salida deseadas?


In [11]:
mensaje_input = "El 25 de Marzo y serian tres noches"
result = await run_agent(agente_reservas, mensaje_input)

Input: El 25 de Marzo y serian tres noches
Tool Call: consultar_reservas({"fecha_salida":"2025-03-28","fecha_entrada":"2025-03-25"})
Tool Output: La habitación está disponible entre las fechas 2025-03-25 y 2025-03-28
Assistant: Tenemos disponibilidad para esas fechas.


In [12]:
mensaje_input = "Del 2025"
result = await run_agent(agente_reservas, mensaje_input)

Input: Del 2025
Assistant: Por favor, especifica las fechas de entrada y salida para las que quieres consultar la disponibilidad.


In [13]:
mensaje_input = "La fecha de entrada seria el 25 de Marzo del 2025 y la fecha de salida seria el 28 de Marzo del 2025"
result = await run_agent(agente_reservas, mensaje_input)

Input: La fecha de entrada seria el 25 de Marzo del 2025 y la fecha de salida seria el 28 de Marzo del 2025
Tool Call: consultar_reservas({"fecha_salida":"2025-03-28","fecha_entrada":"2025-03-25"})
Tool Output: La habitación está disponible entre las fechas 2025-03-25 y 2025-03-28
Assistant: La habitación está disponible entre el 25 y el 28 de Marzo.


In [14]:
# MAKE CHANGES HERE *
# Ejecutamos el agente
mensaje_input = "Mi nombre es Laure M. Quiero hacer una rserva para el 25 de Marzo. Cuatro dias y tres noches."
conversation_history = result.to_input_list()
nuevo_input = conversation_history + [{"role": "user", "content": mensaje_input}]



---



In [15]:
@function_tool
def consultar_reservas(fecha_entrada: date, fecha_salida: date) -> str:

    for reserva in reservas_db:
        # Verificar si hay solapamiento de fechas
        if (fecha_entrada <= reserva["fecha_salida"] and fecha_salida >= reserva["fecha_entrada"]):
            return f"La habitación no está disponible entre las fechas {fecha_entrada} y {fecha_salida}"

    return f"La habitación está disponible entre las fechas {fecha_entrada} y {fecha_salida}"

@function_tool
def crear_reserva(fecha_entrada: date, fecha_salida: date, cliente: str) -> str:
        # Crear nueva reserva
        nueva_reserva = {
            "num_reserva": len(reservas_db) + 1,
            "fecha_entrada": fecha_entrada,
            "fecha_salida": fecha_salida,
            "cliente": cliente
        }
        reservas_db.append(nueva_reserva)
        return f"La reserva ha sido creada con éxito"


today = date.today()

instructions = f"""Eres un asistente inteligente que gestiona reservas para el Hotel del Sol.
El hotel tiene una sola habitación.
Cuando te pidan una reserva, primero comprueba si hay disponibilidad para las fechas indicadas. Hoy es {today}.
Si no hay disponibilidad, devuelve un mensaje indicando que no hay habitaciones disponibles.
Si hay disponibilidad, pide el numero de noches y el nombre del cliente para crear la reserva y devuelve confirmación de la reserva.
"""

# Definimos el agente
agente_reservas = Agent(
    model=MODEL_NAME,
    name="Agente de Reservas",
    instructions=instructions,
    tools=[consultar_reservas, crear_reserva],
)

In [16]:
# Ejecutamos el agente
mensaje_input = "Hola quería reservar una habitación para el 25 de Marzo"
result = await run_agent(agente_reservas, mensaje_input)

Input: Hola quería reservar una habitación para el 25 de Marzo
Assistant: ¿Para cuántas noches le gustaría reservar la habitación?


In [17]:
# Ejecutamos el agente
mensaje_input = "Para 3 noches"
conversation_history = result.to_input_list()
mensajes = conversation_history + [{"role": "user", "content": mensaje_input}]
result = await run_agent(agente_reservas, mensajes)

Input: [{'content': 'Hola quería reservar una habitación para el 25 de Marzo', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '¿Para cuántas noches le gustaría reservar la habitación?\n', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': 'Para 3 noches'}]
Assistant: ¿A nombre de quién hago la reserva?


In [18]:
# Ejecutamos el agente
mensaje_input = "A nombre de Maria Pérez"
conversation_history = result.to_input_list()
mensajes = conversation_history + [{"role": "user", "content": mensaje_input}]
result = await run_agent(agente_reservas, mensajes)

Input: [{'content': 'Hola quería reservar una habitación para el 25 de Marzo', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '¿Para cuántas noches le gustaría reservar la habitación?\n', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': 'Para 3 noches'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '¿A nombre de quién hago la reserva?', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': 'A nombre de Maria Pérez'}]
Tool Call: crear_reserva({"fecha_salida":"2025-03-28","fecha_entrada":"2025-03-25","cliente":"Maria Pérez"})
Tool Output: La reserva ha sido creada con éxito
Assistant: ¡Perfecto, la reserva a nombre de Maria Pérez del 25 de Marzo al 28 de Marzo ha sido creada con éxito!


In [19]:
# Comprobamos que la habitación 1 está reservada
for reserva in reservas_db:
    print(reserva)

{'num_reserva': 1, 'fecha_entrada': datetime.date(2025, 4, 1), 'fecha_salida': datetime.date(2025, 4, 5), 'cliente': 'Juan Pérez'}
{'num_reserva': 2, 'fecha_entrada': datetime.date(2025, 4, 18), 'fecha_salida': datetime.date(2025, 4, 20), 'cliente': 'Pedro López'}
{'num_reserva': 3, 'fecha_entrada': datetime.date(2025, 3, 25), 'fecha_salida': datetime.date(2025, 3, 28), 'cliente': 'Maria Pérez'}


In [20]:
# Prueba de reservar de nuevo con la misma fecha
mensaje_input = "Hola quería reservar una habitación para el 25 de Marzo por 2 noches"
result = await run_agent(agente_reservas, mensaje_input)

Input: Hola quería reservar una habitación para el 25 de Marzo por 2 noches
Tool Call: consultar_reservas({"fecha_salida":"2025-03-27","fecha_entrada":"2025-03-25"})
Tool Output: La habitación no está disponible entre las fechas 2025-03-25 y 2025-03-27
Assistant: No hay habitaciones disponibles para esas fechas.


In [21]:
mensaje_input = "Hola quería reservar una habitación para el 13 de Abril por 2 noches"
result = await run_agent(agente_reservas, mensaje_input)

Input: Hola quería reservar una habitación para el 13 de Abril por 2 noches
Tool Call: consultar_reservas({"fecha_salida":"2025-04-15","fecha_entrada":"2025-04-13"})
Tool Output: La habitación está disponible entre las fechas 2025-04-13 y 2025-04-15
Assistant: La habitación está disponible. Por favor, dime tu nombre para completar la reserva.
